In [0]:
import pyspark.sql.functions as F
df = spark.read.table("ecommerce.bronze.order_base_table")
df = df.withColumn("quantity", F.when(F.col("quantity") == "Two",2).otherwise(F.col("quantity").cast("int")))
df = df.withColumn("unit_price",F.regexp_replace(F.col("unit_price"),"[$]","").cast("double"))
df = df.dropDuplicates(['order_id','item_seq'])
df = df.withColumn("discount_pct",(F.regexp_replace(F.col("discount_pct"), "%", "").cast("double") / 100))
df = df.withColumn("dt",F.to_date(F.col("dt"),"yyyy-MM-dd"))
df = df.withColumn("order_ts",F.to_date(F.col("order_ts"),"yyyy-MM-dd HH:mm:ss"))
df = df.withColumn("item_seq",F.col("item_seq").cast("int"))
df = df.withColumn("tax_amount",F.col("tax_amount").cast("int"))
df = df.withColumn("processed_time",F.current_timestamp())

df.write.format("delta").mode("overwrite").saveAsTable("ecommerce.silver.order_base_table")